# Predictive IT Infrastructure Management: CPU Usage Forecasting

**Copyright (c) 2026 Shrikara Kaudambady**

This notebook demonstrates a machine learning approach for a critical IT infrastructure management task: **resource forecasting**. Specifically, we will predict future server CPU utilization based on historical data.

**Problem:** Without accurate forecasting, IT teams either over-provision resources (wasting money) or under-provision them (risking performance degradation and outages). 
**Solution:** We will use `Prophet`, a powerful time-series forecasting library developed by Facebook, to build a model that learns trends and seasonal patterns from past CPU usage to predict future demand.

## 1. Setup and Library Imports

We import the necessary libraries. The key library here is `prophet`.

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt

# Suppress warnings from Prophet
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

## 2. Data Generation

For this demonstration, we will generate a synthetic dataset that mimics realistic server CPU utilization over several months. This data will include:
- A base utilization level.
- A slight upward trend (representing growing service demand).
- Daily seasonality (lower usage at night).
- Weekly seasonality (lower usage on weekends).
- Some random noise.

In [ ]:
def generate_cpu_data(days=180, freq='H'):
    timestamps = pd.date_range(start='2025-07-01', periods=days * 24, freq=freq)
    n_points = len(timestamps)
    
    # Base and trend
    base_cpu = 20
    trend = np.linspace(0, 15, n_points) # CPU usage grows by 15% over the period
    
    # Seasonalities
    daily_seasonality = 10 * np.sin(2 * np.pi * timestamps.hour / 24)
    weekly_seasonality = 5 * (1 - np.isin(timestamps.dayofweek, [5, 6])) # Lower on weekends (Sat/Sun)
    
    # Noise
    noise = np.random.normal(0, 2, n_points)
    
    # Combine components
    cpu_usage = base_cpu + trend + daily_seasonality + weekly_seasonality + noise
    cpu_usage = np.clip(cpu_usage, 5, 100) # Clip values to be within a realistic range
    
    df = pd.DataFrame({'ds': timestamps, 'y': cpu_usage})
    return df

# Generate the data
df = generate_cpu_data()

print("Generated Data Shape:", df.shape)
df.head()

## 3. Data Exploration

Let's visualize the generated time-series data to see the patterns we've created.

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(df['ds'], df['y'])
plt.title('Synthetic CPU Utilization Over Time')
plt.xlabel('Date')
plt.ylabel('CPU Usage (%)')
plt.grid(True)
plt.show()

## 4. Training the Forecasting Model

Training a model with Prophet is straightforward. It expects the data to be in a DataFrame with two columns:
- `ds`: The datestamp column (must be datetime format).
- `y`: The numeric value we want to forecast.

Our DataFrame is already in this format.

In [ ]:
# Initialize and fit the model
# Prophet will automatically detect the daily and weekly seasonalities
model = Prophet(daily_seasonality=True, weekly_seasonality=True)
model.fit(df)

## 5. Making a Future Forecast

We'll instruct the model to forecast the CPU utilization for the next 30 days.

In [ ]:
# Create a future dataframe for the next 30 days
future = model.make_future_dataframe(periods=30*24, freq='H')

# Generate the forecast
forecast = model.predict(future)

print("Forecast Data Columns:")
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail())

## 6. Visualizing the Forecast Results

Prophet has excellent built-in plotting capabilities. The first plot shows the original data (black dots), the model's forecast (`yhat` in blue), and the uncertainty interval (light blue shaded area).

In [ ]:
fig1 = model.plot(forecast, figsize=(15, 6))
plt.title('CPU Utilization Forecast for the Next 30 Days')
plt.xlabel('Date')
plt.ylabel('CPU Usage (%)')
plt.show()

## 7. Analyzing Forecast Components

This is one of the most powerful features of Prophet. We can decompose the forecast to see the individual components the model has learned:
- **Trend:** The overall direction of the data.
- **Weekly Seasonality:** The pattern of usage over a week.
- **Daily Seasonality:** The pattern of usage over a day.

In [ ]:
fig2 = model.plot_components(forecast, figsize=(15, 10))
plt.show()

## 8. Conclusion and Practical Application

We have successfully built a forecasting model that can predict future CPU demand with reasonable accuracy. The component plots clearly show that the model has learned the underlying patterns in our data.

**How an IT Manager would use this:**
1.  **Capacity Planning:** By looking at the upward trend in the forecast, a manager can see that CPU usage is projected to increase. The upper bound of the forecast (`yhat_upper`) can be used as a threshold to decide when to provision new hardware or allocate more virtual resources.
2.  **Anomaly Detection:** If the *actual* CPU usage significantly exceeds the forecasted `yhat_upper`, it could trigger an alert, indicating an unexpected problem that requires investigation.
3.  **Maintenance Planning:** The seasonality plots show that usage is lowest overnight and on weekends. This information can be used to schedule maintenance and updates during periods of low impact.